In [1]:
%load_ext watermark
%watermark

2018-09-20T11:17:28-05:00

CPython 3.6.5
IPython 6.4.0

compiler   : MSC v.1900 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 78 Stepping 3, GenuineIntel
CPU cores  : 4
interpreter: 64bit


In [171]:
!python -m cpppo.server.enip.client --udp --broadcast --list-identity -a 192.168.1.35

List Identity  0 from ('192.168.1.35', 44818): {
    "count": 1,
    "item[0].type_id": 12,
    "item[0].length": 56,
    "item[0].identity_object.version": 1,
    "item[0].identity_object.sin_family": 2,
    "item[0].identity_object.sin_port": 44818,
    "item[0].identity_object.sin_addr": "192.168.1.35",
    "item[0].identity_object.vendor_id": 1,
    "item[0].identity_object.device_type": 12,
    "item[0].identity_object.product_code": 200,
    "item[0].identity_object.product_revision": 1802,
    "item[0].identity_object.status_word": 48,
    "item[0].identity_object.serial_number": 12174578,
    "item[0].identity_object.product_name": "1756-EN2TR/C 217021900",
    "item[0].identity_object.state": 3
}


In [4]:
from cpppo.server.enip.get_attribute import proxy_simple
product_name, = proxy_simple( "192.168.1.35" ).read( [('@1/1/7','SSTRING')] )
product_name

['1756-EN2TR/C 217021900']

In [72]:
from cpppo.server.enip.get_attribute import proxy
via = proxy( "192.168.1.35" )

with via:
    vendor, product_name = via.read( [('@1/1/1','INT'), ('@1/1/7','SSTRING')] )
    
print(product_name[0])

1756-L71/B LOGIX5571


In [78]:
from cpppo.server.enip.client import connector
from cpppo.server.enip import client
host = "192.168.1.35"

tags = ["RecetasPL[0].Medida.LEN"]

with connector( host=host ) as conn:
    for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2):
        val = str(value[0])
        valor = "RecetasPL[0].Medida.Data[0-"+val+"]"

RecetasPL[0].Medida.Data[0-7]


In [62]:
from cpppo.server.enip.client import connector
from cpppo.server.enip import client
host = "192.168.1.35"

tags = ["RecetasPL[0].Medida.LEN"]

with connector( host=host ) as conn:
    for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2):
        valor = "RecetasPL[0].Medida.Data"+str(value)
        print(valor)

RecetasPL[0].Medida.Data[7]


In [71]:
from cpppo.server.enip.client import connector
from cpppo.server.enip import client
host = "192.168.1.35"

tags = ["RecetasPL[0].Medida.Data[0-7]"]

with connector( host=host ) as conn:
    for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2):
        print( "{} | {}".format( descr, ''.join(chr(i) for i in value) ))

Single Read  Tag  RecetasPL.Medida.Data[0-7] | PL70156 


In [68]:
from cpppo.server.enip.client import connector
from cpppo.server.enip import client
host = "192.168.1.35"
tags = ["RecetasPL[0].VelocidadMax","RecetasPL[0].MedidaNumElementos=(DINT)200"]

with connector( host=host ) as conn:
    for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2 ):
        print( "{} | {}".format( descr, value ))

Single Read  Tag  RecetasPL.VelocidadMax[0] | [17]
Single Write Tag  RecetasPL.MedidaNumElementos[0-0] | True


In [70]:
from cpppo.server.enip.client import connector
from cpppo.server.enip import client
host = "192.168.1.35"
tags = ["RecetasPL[0].VelocidadMax","RecetasPL[0].MedidaNumElementos"]

with connector( host=host ) as conn:
    for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2 ):
        print( "{} | {}".format( descr, value ))

Single Read  Tag  RecetasPL.VelocidadMax[0] | [17]
Single Read  Tag  RecetasPL.MedidaNumElementos[0] | [200]


In [161]:
def leerString(host,indice,etiqueta):
    from cpppo.server.enip.client import connector
    from cpppo.server.enip import client
    
    tags = ["RecetasPL[{}].{}.LEN".format(indice,etiqueta)]
    
    with connector( host=host ) as conn:
        for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2):
            val = str(value[0])
            valor = ["RecetasPL[0].Medida.Data[0-"+val+"]"]
            with connector( host=host ) as conn:
                for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( valor ), depth=2):
                    final = ''.join(chr(i) for i in value).strip("\x00")
                    return final

var = leerString("192.168.1.35",0,"Medida")
print(var)
var

PL70156


'PL70156'

In [173]:
def DINT(host, tags):
    from cpppo.server.enip.client import connector
    from cpppo.server.enip import client
    
    with connector( host=host ) as conn:
        for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2 ):
            if value == True or value == False:
                yield value
            else:
                yield value[0]
                
tags = ["RecetasPL[0].VelocidadMax=(DINT)20","RecetasPL[0].MedidaNumElementos"]
datos = DINT("192.168.1.35", tags)
for i in zip(tags, datos):
    print(i)
    
print("=============================================================")

tags = ["RecetasPL[0].VelocidadMax","RecetasPL[0].MedidaNumElementos"]
datos = DINT("192.168.1.35", tags)
for i in zip(tags, datos):
    print(i)

('RecetasPL[0].VelocidadMax=(DINT)20', True)
('RecetasPL[0].MedidaNumElementos', 200)
('RecetasPL[0].VelocidadMax', 20)
('RecetasPL[0].MedidaNumElementos', 200)


In [169]:
for i in range(0,299):
    var = leerString("192.168.1.35",i,"Medida")
    if len(var) > 1:
        print(var)
    else:
        break

PL70156
PL70156
PL70156
PL70156
